In [1]:
#import dependencies
import datetime as dt
##import numpy as np
import pandas as pd

In [2]:
#dependencies we need for SQLAlchemy
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
#import Flask dependencies 
from flask import Flask, jsonify

In [4]:
#Access to SQLite Database
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
#reflect the database into our classes
Base = automap_base()

In [6]:
#Add the following code to reflect the database
Base.prepare(engine, reflect=True)

In [7]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station
#session link from Python to our database
session = Session(engine)

In [8]:
#Define the Flask Application
app = Flask(__name__)

In [9]:
#Creating a Flask Route
@app.route("/")
def welcome():
    return(
    '''
    Welcome to the Climate Analysis API!
    Available Routes:
    /api/v1.0/precipitation
    /api/v1.0/stations
    /api/v1.0/tobs
    /api/v1.0/temp/start/end
    ''')